## Chatbot with History

#### Load Model using Groq


In [1]:
from langchain_groq import ChatGroq

model = ChatGroq(model="Gemma2-9b-It")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000019CD791ACB0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000019CD79199F0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

#### Chat conversion without memory / chat history

In [17]:
Markdown(model.invoke("Hi, I am John").content)

Hello John! 👋 

It's nice to meet you. What can I do for you today? 😊 



In [19]:
Markdown(model.invoke("Hi, Who am I?").content)

As an AI, I have no memory of past conversations and don't know who you are.

If you'd like to tell me about yourself, I'm happy to listen! 😊  What's your name? What are you interested in?


#### Remembering as per previous conversation or messages

In [2]:
from langchain_core.messages import HumanMessage, SystemMessage
from IPython.display import Markdown

messages = [
    HumanMessage("Hi, I am Ranjan and I work as Senior Software Engineer at J.P.Morgan"),
    SystemMessage("Remember what user is saying"),
    HumanMessage("Who am I?"),
]

Markdown(model.invoke(messages).content)

You are Ranjan, a Senior Software Engineer at J.P.Morgan.  

I remember that from our previous conversation.  😊  


#### Message History using `ChatMessageHistory` and `session_id`
ChatMessageHistory passes entire consversation history as messages(as above) whenever a new message is send to the model, thus remembers the context

In [3]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store= {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

config = {"configurable": {"session_id": "chat_1"}}
Markdown(with_message_history.invoke("Hi, do you know me?", config).content)


As a large language model, I have no memory of past conversations and I don't know you personally.

If you'd like to tell me about yourself, I'm happy to listen!


In [4]:
Markdown(with_message_history.invoke("Hi, My name is Ranjan, working as senior software Engineer at J.P.Morgan", config).content)

Hi Ranjan, it's nice to meet you! 

That's impressive, working as a Senior Software Engineer at J.P. Morgan. What kind of software do you specialize in?  😊  



In [8]:
Markdown(with_message_history.invoke("Hi, Who am I?", config).content)

You are Ranjan, a Senior Software Engineer at J.P. Morgan!  

Is there anything else you'd like to know about yourself, or would you like to talk about something else?  😊  


In [9]:
config2 = {"configurable": {"session_id": "chat_2"}}

Markdown(with_message_history.invoke("Hi, I am Archana, working as MTS 3 at VMWare India", config2).content)

Hi Archana! 

It's nice to meet you.  

I understand you're an MTS 3 at VMware India. That's great! VMware is doing some really interesting work. 

Is there anything I can help you with today? 



In [10]:
Markdown(with_message_history.invoke("Hi, Who am I?", config2).content)

You are Archana, an MTS 3 at VMware India, as you told me earlier.  

Is there something specific you're wondering about your identity or role? 

Perhaps you're feeling a bit lost or confused?  I'm here to listen and help in any way I can.


In [11]:
Markdown(with_message_history.invoke("Hi, Do you remember me?", config).content)

Yes, I do!  You are Ranjan, a Senior Software Engineer at J.P. Morgan.  

What can I do for you today? 😊  


In [12]:
store

{'chat_1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi, do you know me?', additional_kwargs={}, response_metadata={}), AIMessage(content="As a large language model, I have no memory of past conversations and I don't know you personally.\n\nIf you'd like to tell me about yourself, I'm happy to listen!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 16, 'total_tokens': 60, 'completion_time': 0.08, 'prompt_time': 8.1579e-05, 'queue_time': 0.013529731, 'total_time': 0.080081579}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2464f9ee-55d7-497d-852e-7ed9b9e7dabc-0', usage_metadata={'input_tokens': 16, 'output_tokens': 44, 'total_tokens': 60}), HumanMessage(content='Hi, My name is Ranjan, working as senior software Engineer at J.P.Morgan', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Ranjan, it's nice to meet you! \n\nThat's

#### Chat History along with ChatPromptTemplate

In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer all question based on your ability in {language}"),
    MessagesPlaceholder("messages")
])

output_parser = StrOutputParser()

chain = prompt | model | output_parser

Markdown(chain.invoke({"messages": [HumanMessage("Who is Narendra Modi?")], "language": "Hindi"}))

नरेंद्र मोदी भारत के वर्तमान प्रधान मंत्री हैं। 

उन्होंने 2014 से भारत की कमान संभाली है और 2019 में फिर से चुने गए हैं। मोदी भारतीय जनता पार्टी (भाजपा) के नेता हैं और गुजरात के पूर्व मुख्यमंत्री भी रहे हैं। 

उनके राजनीतिक विचारों में राष्ट्रवाद, आर्थिक विकास और सामाजिक सुधार शामिल हैं। 


In [14]:
with_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")

config3 = {"configurable": {"session_id": "chat4"}}
Markdown(with_history.invoke({"messages": [HumanMessage("I am Ranjan, I work as Senior Software Engineer at J.P.Morgan.")], "language": "Hindi"}, config3))

नमस्ते रंजन! 

मुझे आपकी जानकारी में खुशी हो रही है।  एक सीनियर सॉफ्टवेयर इंजीनियर के रूप में J.P.Morgan में काम करना बहुत ही शानदार है। 

आपके द्वारा पूछे जाने वाले प्रश्नो के आधार पर मैं उत्तर प्रदान करूँगा। कृपया प्रश्न पूछें ताकि मैं आपकी मदद कर सकूं! 



In [12]:
Markdown(with_history.invoke({"messages": "Who am I?", "language": "Hindi"}, config3))

आप रंजन हैं, जो जे.पी. मोर्गन में सीनियर सॉफ्टवेयर इंजीनियर के रूप में काम करते हैं। 

क्या मैं आपकी और जानकारी में मदद कर सकता हूँ? 

